# Movimento Vertical (Heave) de um cilindro flutuante

Autor: Daniel Prata Vieira

1. Realizar o cálculo do período natural de um cilindro com diâmetro 𝐷 e calado 𝑇
1. Obter a curva de decaimento para 3 amplitudes iniciais: 1\%, 2\% e 5\% do calado

Dados:
- $[𝐷_1,𝐷_2, 𝐷_3]$ = três últimos dígitos do n° usp
- $D = 10 + D_1$ [m]
- $T = 15 + D_2$ [m]
- $\zeta = 2 + D_3$ [\%]

- $m_a = \frac{2}{3} \pi \rho (\frac{D}{2})^3$ [kg]

## Equação do movimento


$x(t)$ : movimento vertical do cilindro




> **Energia cinética**
>
> $T = \dfrac{1}{2} \cdot m \cdot \dot{x}^2$

> **Energia Potencial**
>
> $U = \dfrac{1}{2} \cdot k \cdot x^2$

> **Força Não-Conservadora (Amortecimento)**
>
> $Q'_x = F_d = -c \cdot \dot{x}$


In [1]:
from sympy import latex
from IPython.display import display, Math

def print_sym(s):
    display(Math(latex(s)))

## Dedução das equações utilizando cálculo simbólico

In [2]:
import sympy as sym
sym.init_printing()

# Definindo as variáveis
t = sym.symbols('t')
m = sym.symbols('m')
k = sym.symbols('k')
c = sym.symbols('c')
x = sym.Function('x')(t)

# Deduzindo a equação
xp = sym.diff(x, t)
xpp = sym.diff(x, t, 2)

# Energia Cinética
T = 1/2  * m * (xp**2)

# Energia Potencial
U = 1/2 * k * (x**2)

# Força Dissipativa
Qx = - c * xp

> **Lagrangeana**
>
> $L = T - U = \dfrac{1}{2} \cdot m \cdot \dot{x}^2 - \dfrac{1}{2} \cdot k \cdot x^2$

In [3]:
L = T - U
print_sym(L)

<IPython.core.display.Math object>

**Cálculo dos Termos da Equação:**

* $\dfrac{\partial L}{\partial x}$

* $\dfrac{\partial L}{\partial \dot{x}}$

* $ \dfrac{d}{dt} \left( \dfrac{\partial L}{\partial \dot{x}} \right)$






In [4]:
dLdx = sym.diff(L, x)

dLdxp = sym.diff(L, xp)

d_dLdxp_dt = sym.diff(dLdxp, t)


**Equação do Movimento**

$ \dfrac{d}{dt} \left( \dfrac{\partial L}{\partial \dot{x}} \right) - Q'_x - \dfrac{\partial L}{\partial x} = 0$

In [5]:
eq = d_dLdxp_dt - Qx - dLdx
print_sym(eq)

<IPython.core.display.Math object>

## Simulação Numérica

In [6]:
import numpy as np

rho = 1000 #kg/m³
g   = 9.81   #m/s²

np.random.seed(100)
D1, D2, D3 = np.random.randint(0, 10, size=3)
print(f'D1 = {D1}; D2 = {D2}; D3 = {D3}')

D = 10 + D1
T = 15 + D2
zeta = (2+D3)/100
ma = (2/3)*np.pi*rho*((D/2)**3)
m = rho * T * (np.pi * D**2) /4

print(f'Diâmetro = {D} m')
print(f'Calado = {T} m')
print(f'Amortecimento = {zeta*100}%')
print(f'Massa = {m/1000:.2f} t')
print(f'Massa Adicional = {ma/1000:.2f} t')

D1 = 8; D2 = 8; D3 = 3
Diâmetro = 18 m
Calado = 23 m
Amortecimento = 5.0%
Massa = 5852.79 t
Massa Adicional = 1526.81 t


## Frequência Natual

$\omega_n$ : Frequência Natural em rad/s

$\omega_n = \sqrt{\dfrac{\rho g A_{WL}}{m+m_a}}$

In [7]:
Awl = (np.pi * D**2)/4

k = rho*g*Awl

wn = np.sqrt(k/(m + ma))
Tn = 2*np.pi/wn

print(f'wn = {wn:.4f} rad/s')

print(f'Tn = {Tn:.2f} s')

w = wn

wn = 0.5816 rad/s
Tn = 10.80 s


## Amortecimento Crítico

$c_{crítico} = 2 \cdot \omega_n \cdot (m + m_a)$

$c_{crítico} = 2 \cdot \sqrt{ k \cdot (m + m_a)}$

In [8]:
c_crit = 2*wn*(m + ma)

print('c_crit = ' + str(c_crit) + ' Ns/m')

c = zeta * c_crit

print('c = ' + str(c) + ' Ns/m')

c_crit = 8584171.583454851 Ns/m
c = 429208.57917274255 Ns/m


## Equacionamento

Isolar o termo de aceleração:

$ \ddot x  = \dfrac{1}{(m + m_a)} (- c \cdot \dot x  - k \cdot x )$

Inserir uma nova variável $v$, tal que:

 $\dot x = v$     (Eq. 1)
 
 temos:

$\dot v = \dfrac{1}{(m + m_a)} (- c \cdot v  - k \cdot x) $ (Eq. 2)

Assumindo a variável $f$, tal que:

$f = \begin{bmatrix}x \\ v\end{bmatrix}$

Tem-se:

$\dot f = \begin{bmatrix}\dot x \\ \dot v\end{bmatrix} = \begin{bmatrix}v \\ \dfrac{1}{(m + m_a)} (- c \cdot v  - k \cdot x )\end{bmatrix}$


$\dot f = F(t, f)$

In [9]:
# equações a serem resolvidas

def eq1(t, x, v):
  xp = v
  return xp

def eq2(t, x, v):
  vp = (-c*v - k*x)/(m + ma) # Equação Característica
  return vp

def eq_vetorial(t, f):
  dxdt = eq1(t, f[0], f[1])
  dvdt = eq2(t, f[0], f[1])

  dfdt = [dxdt, dvdt]

  return dfdt

# Solver Numérico (Runge-Kutta)

In [10]:
from scipy.integrate import solve_ivp

# Condições Iniciais
X0 = (1/100)*T
#X0 = (2/100)*T
#X0 = (5/100)*T
print(f'Amplitude inicial = {X0} m')

# Definição do vetor de tempo
f0 = np.array([X0, 0])
t_inicial = 0 #s
t_final = 100  #s
N_pts = 5000
t = np.linspace(t_inicial, t_final, N_pts)

f_res = solve_ivp(eq_vetorial, (t_inicial, t_final), f0, t_eval=t, max_step=5e-3)

Amplitude inicial = 0.23 m


# Visualização dos resultados

In [11]:
# Formato do vetor de resultados
np.shape(f_res.y)

In [12]:
import plotly.graph_objects as go

# Cria a figura e adiciona a curva
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=f_res.y[0,:], name='y(t)'))

# Define o título do gráfico e dos eixos
fig.update_layout(title='Simulação Decaimento', xaxis_title='tempo (s)', yaxis_title='y(t)')

# Exibe o gráfico
fig.show()


**Envoltória**

$y_{env} = X_0 \cdot e^{-\zeta \cdot \omega_n \cdot t}$

In [13]:
from scipy.signal import find_peaks
picos, _ = find_peaks(f_res.y[0,:])
vales, _ = find_peaks(-f_res.y[0,:])

# Envoltória
x_env = X0*np.exp(-zeta*wn*t)

#Definição de marcadores
marker01 = dict(color='red', symbol='triangle-up', size=9)
marker02 = dict(color='green', symbol='triangle-down', size=9)
line1 = dict(color='orange', width=2, dash='dash')
line2 = dict(color='magenta', width=2, dash='dash')

# Cria a figura e adiciona as curvas
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=f_res.y[0,:], name='x(t)'))
fig.add_trace(go.Scatter(x=t[picos], y=f_res.y[0,picos], name='Picos', mode='markers', marker=marker01))
fig.add_trace(go.Scatter(x=t[vales], y=f_res.y[0,vales], name='Vales', mode='markers', marker=marker02))
fig.add_trace(go.Scatter(x=t, y=x_env, name='envoltoria superior', line=line1))
fig.add_trace(go.Scatter(x=t, y=-x_env, name='envoltoria inferior', line=line2))

# Define o título do gráfico e dos eixos
fig.update_layout(title='Simulação Decaimento', xaxis_title='tempo (s)', yaxis_title='x(t)')
fig.show()



# Diagrama de fase

In [14]:
# plt.figure(figsize=(5,5), dpi= 100, facecolor='w', edgecolor='k')
# plt.plot(f_res.y[0,:], f_res.y[1,:])
# plt.ylabel('xp (t) [m/s]')
# plt.xlabel('x (t) [m]')
# plt.grid()

# Cria a figura e adiciona a curva
fig = go.Figure()
fig.add_trace(go.Scatter(x=f_res.y[0,:], y=f_res.y[1,:], name='diag. fase'))

# Define o título do gráfico e dos eixos
fig.update_layout(title='Diagrama de Fase', xaxis_title='x (t) [m]', yaxis_title='xp (t) [m/s]', width=600, height=600)
fig.show()